# Interschulz Crawl

셀레니움 설치

In [ ]:
pip install selenium

제품 & 기업 페이지 1page 부터 201page까지 html만 soup해서 리스트에 저장

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from tqdm import tqdm

# 속도 향상을 위한 이미지 로딩 제거
chromedriver = 'C:\Code\Webdriver\chromedriver.exe' # 크롬드라이버 경로
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)

action = ActionChains(driver)
# 쿠키 없이 들어가면 아무정보도 안뜸
driver.get('https://www.interschutz.de/en/')
time.sleep(1)

driver.get('https://www.interschutz.de/en/search/?category=ep') # 배너클릭해서 들어가는것보다 그냥 집어넣어주는게 효율적이고 쉬움

soup_list = []
for i in tqdm(range(202-1)):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    soup_list.append(soup)
    bottom = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-shrink > section.c.footer.is-dark.module-theme-400")
    action.move_to_element(bottom).perform()

    # Selector를 뭘 쓰느냐에 따라서 소요시간이 달라짐 ex) Xpath가 찾는데 쥰내오래걸림
    try:
        driver.find_element(By.CSS_SELECTOR,'#search-results-reveal > div > div.reveal-master > div > div > div.cell.search-results-list.medium-8 > section > div > div.grid-x.search-snippet-list-footer > div:nth-child(2) > section > a:nth-child(8)').click()
    except:
        driver.find_element(By.CSS_SELECTOR,'#search-results-reveal > div > div.reveal-master > div > div > div.cell.search-results-list.medium-8 > section > div > div.grid-x.search-snippet-list-footer > div:nth-child(2) > section > a:nth-child(7)').click()
    time.sleep(1)

driver.quit()

<ipython-input-3-a5b14c13f463>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
<ipython-input-3-a5b14c13f463>:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)


저장된 soup에서 제품만 분리, 페이지 URL 추출

In [4]:
link_list = []
for i in soup_list:
    temp_list = i.find_all('a','o link as-block fx dropshadow for-child')
    for l in temp_list:
        link_list.append(l.attrs['href'])

prd_link_list = []
for i in link_list:
    switch = i.split("/")[1]
    if switch == 'product':
        prd_link_list.append(i)

import requests
import pandas as pd
from tqdm import tqdm

prd_soup_list = [] # 제품만 긁어도 됨
prd_full_list = []

base_url = "https://www.interschutz.de/"

for i in tqdm(prd_link_list):
    switch = i.split("/")[1]
    response = requests.get(base_url+i)
    prd_full_list.append(base_url+i)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    prd_soup_list.append(soup)



 15%|█▌        | 411/2668 [06:08<39:16,  1.04s/it] 

저장된 페이지 URL에 접속해 크롤링 데이터 수집

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
# 속도 향상을 위한 이미지 로딩 제거
chromedriver = 'C:\Code\Webdriver\chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
action = ActionChains(driver)

v1_list = []
v2_list = []
v3_list = []
v4_list = []
v5_list = []
v6_list = []

base_url = "https://www.interschutz.de/"

for i in tqdm(prd_link_list):
    url = base_url+i
    # URL 접속
    driver.get(url)

    # 제품명
    try:
        v1 = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.t.set-800-bold > h1").text
    except:
        v1 = np.nan
    # 제조사
    try:
        v2 = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.page-intro-description.with-function-bar > div.grid-x.grid-padding-x > div.cell.auto > div:nth-child(1)").text
    except:
        v2 = np.nan
    # 설명
    try:
        try:
            v3 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(2) > div > div > div > div > p").text
        except:
            v3 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > p").text
    except:
        v3 = np.nan
    # 제품그룹
    try:
        v4 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > ul").text.split("\n")
    except:
        try:
            v4 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(2) > div > div > div > div > ul").text.split("\n")
        except:
            v4 = np.nan

    # 제조사명
    v5 = driver.find_element(By.CSS_SELECTOR,"#tabs-top > div.tabs-track > div > div.is-active > div > div > div:nth-child(1) > h2").text
    try:
        v6 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > a").get_attribute('href')
    except:
        v6 = np.nan

    v1_list.append(v1)
    v2_list.append(v2)
    v3_list.append(v3)
    v4_list.append(v4)
    v5_list.append(v5)
    v6_list.append(v6)
        
driver.quit()

<ipython-input-153-c3e9526b4492>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
<ipython-input-153-c3e9526b4492>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
100%|██████████| 2668/2668 [1:36:17<00:00,  2.17s/it]


저장

In [ ]:
df = pd.DataFrame({"v1":v1_list,
            "v2":v2_list,
            "v3":v3_list,
            "v4":v4_list,
            "v5":v5_list,
            "v6":v6_list,
            "v7":prd_full_list})

df.to_csv("prd_crawl.csv")

# Test before iteration

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
import numpy as np
# 속도 향상을 위한 이미지 로딩 제거
chromedriver = 'C:\Code\Webdriver\chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
action = ActionChains(driver)

v1_list = []
v2_list = []
v3_list = []
v4_list = []
v5_list = []
v6_list = []

base_url = "https://www.interschutz.de/"
url = base_url+i
url = "https://www.interschutz.de//product/ai-fire-smoke-sensing-network-camera/308581/N1486168"
url = "https://www.interschutz.de//product/deck-integrated-fire-fighting-systems/296646/N1423333"
# URL 접속
driver.get(url)

# 제품명
v1 = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.t.set-800-bold > h1").text

# 제조사
v2 = driver.find_element(By.CSS_SELECTOR,"#app > div.flex-container.flex-dir-column.fullheight > div.flex-child-grow > div > header > div > div > div.cell.small-12.page-intro-content.medium-10.large-9 > div.page-intro-description.with-function-bar > div.grid-x.grid-padding-x > div.cell.auto > div:nth-child(1)").text

# 설명
try:
    try:
        v3 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(2) > div > div > div > div > p").text
    except:
        v3 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > p").text
except:
    v3 = np.nan
# 제품그룹
try:
    v4 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > ul").text.split("\n")
except:
    try:
        v4 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(2) > div > div > div > div > ul").text.split("\n")
    except:
        v4 = np.nan

# 제조사명
v5 = driver.find_element(By.CSS_SELECTOR,"#tabs-top > div.tabs-track > div > div.is-active > div > div > div:nth-child(1) > h2").text
try:
    v6 = driver.find_element(By.CSS_SELECTOR,"#detailProductId1 > div:nth-child(3) > div > div > div > div > a").get_attribute('href')
except:
    v6 = np.nan

v1_list.append(v1)
v2_list.append(v2)
v3_list.append(v3)
v4_list.append(v4)
v5_list.append(v5)
v6_list.append(v6)


<ipython-input-149-5f4d3ba220c5>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
<ipython-input-149-5f4d3ba220c5>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
